In [10]:
using Random
using Statistics
using BenchmarkTools
nx = ny = 200
rng = MersenneTwister(1234)
function make_uvw(nx, ny, rng)
    u = rand(rng, nx, ny)
    v = rand(rng, nx, ny)
    w = similar(u)
    alpha = rand(rng)
    return u, v, w, alpha
end

make_uvw (generic function with 1 method)

In [3]:
function calc_ij(u, v, w, alpha)
    for i in 1:size(u, 1)
        for j in 1:size(u, 2)
            w[i,j] = u[i,j] + v[i,j] * alpha
        end
    end
end
function calc_ji(u, v, w, alpha)
    for j in 1:size(u, 2)
        for i in 1:size(u, 1)
            w[i,j] = u[i,j] + v[i,j] * alpha
        end
    end
end
@benchmark calc_ij(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  57.186 μs …  1.140 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     82.626 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   98.908 μs ± 42.495 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       █▆                                                      
  ▁▁▂▃████▅▄▄▃▃▂▂▂▂▂▂▂▂▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  57.2 μs         Histogram: frequency by time         267 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [4]:
@benchmark calc_ji(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  16.833 μs … 658.389 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.059 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.869 μs ±  33.058 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▆█▇▄▃▃▃▃▂▂▁▂▂▂▁▁      ▂▂▂▁                                 ▂
  ███████████████████▇█▇▇█████████▇▇▇▇▆▆▆▅▆▆▆▅▆▆▆▆▅▄▁▅▅▄▄▄▄▅▄▄ █
  16.8 μs       Histogram: log(frequency) by time       184 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [16]:
function calc_matrix(u, v, w, alpha)
    w = u + v * alpha
    return 0
end
# @benchmark calc_matrix(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))
@benchmark calc_matrix(u, v, w, alpha)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  48.802 μs …   2.316 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):     67.791 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   94.274 μs ± 116.538 μs  ┊ GC (mean ± σ):  15.32% ± 12.49%

  ▆█▆▄▄▂▁   ▁▂▁                                                ▂
  ██████████████▇▇▆▆▄▅▅▄▃▃▃▃▄▅▄▁▁▄▄▃▄▄▄▃▁▁▄▃▄▃▅▆▇▇▆▅▆▄▅▆▆▅▆▇▅▆ █
  48.8 μs       Histogram: log(frequency) by time       745 μs <

 Memory estimate: 625.17 KiB, allocs estimate: 6.

In [15]:
function calc_broadcast(u, v, w, alpha)
    w .= u .+ v .* alpha
    return 0
end
@benchmark calc_broadcast(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  14.797 μs …  1.175 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.379 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.950 μs ± 27.929 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▃▆██▇▅▃▂▂▂▂▂▂▂▃▂▂▁               ▁▁▁                    ▂
  ▇████████████████████████▇▇▇▇▅▆▆▆▆▆▅▇████▇▇▆▆▆▆▇▆▇▆▆▆▆▅▅▆▄▅ █
  14.8 μs      Histogram: log(frequency) by time       127 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [88]:
@benchmark (@. calc_matrix(u, v, w, alpha)) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  11.145 μs …  1.607 ms  ┊ GC (min … max):  0.00% … 93.61%
 Time  (median):     27.102 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   41.154 μs ± 71.826 μs  ┊ GC (mean ± σ):  15.16% ±  8.86%

    █▄▄▅▃▂▁ ▁▁▁▁                                              ▁
  ▅███████████████▆▆▅▅▄▄▄▅▄▄▆▅▆▅▃▃▃▃▃▃▃▃▃▁▃▃▁▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▃ █
  11.1 μs      Histogram: log(frequency) by time       371 μs <

 Memory estimate: 312.59 KiB, allocs estimate: 3.

In [84]:
function calc_eachindex(u, v, w, alpha)
  @inbounds @simd for ij in eachindex(u)
      w[ij] = u[ij] + v[ij] * alpha
  end
end
@benchmark calc_eachindex(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 3 evaluations per sample.
 Range (min … max):  11.506 μs … 345.546 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.949 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.102 μs ±  10.596 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▄▇█▇▅▃▃▃▃▃▃▃▃▃▂▂▁▁                                         ▂
  ▆████████████████████▇█▇█▇▇▇▇▇▆▆▆▆▅▅▄▆▅▆▄▄▆▄▅▅▃▃▂▃▄▄▃▅▃▄▃▄▄▄ █
  11.5 μs       Histogram: log(frequency) by time      66.5 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [ ]:
function calc_cartesian(u, v, w, alpha)
  for ij in CartesianIndices(u)
      # (i, j) = Tuple(ij)
      w[ij] = u[ij] + v[ij] * alpha
  end
end
@benchmark calc_cartesian(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 3 evaluations per sample.
 Range (min … max):  11.282 μs … 206.239 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.568 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.570 μs ±   7.030 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▁▃▅▇██▇▆▄▃▃▃▂▂▃▂▂▂▁ ▁     ▁▁                              ▂
  ▅▇█████████████████████████████████▇▇▇▆▇▇▆▆▆▅▅▆▅▄▅▆▄▅▅▅▅▅▅▄▄ █
  11.3 μs       Histogram: log(frequency) by time      49.1 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [26]:
function calc_linearcartesian(u, v, w, alpha)
  @inbounds @simd for ij in LinearIndices(CartesianIndices(u))
      w[ij] = u[ij] + v[ij] * alpha
  end
end
@benchmark calc_linearcartesian(u, v, w, alpha) setup=((u, v, w, alpha) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 3 evaluations per sample.
 Range (min … max):  12.371 μs …  1.780 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.604 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.775 μs ± 20.719 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▅▇█▇▆▄▄▄▃▃▃▃▃▃▂▂▁▁   ▁▁▂▁                                 ▂
  ▄▇████████████████████████████████▇█▆▇▇▆▇▆▇▇▅▆▄▅▅▄▄▅▄▄▅▄▂▃▂ █
  12.4 μs      Histogram: log(frequency) by time      70.2 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [77]:
w = similar(u)
println("address of w: ", pointer(w))
w = u + v
println("address of w: ", pointer(w))
w = u .+ v
println("address of w: ", pointer(w))
w .= u + v
println("address of w: ", pointer(w))
w .= u .+ v
println("address of w: ", pointer(w))
w .= @. u + v
println("address of w: ", pointer(w))
broadcast(+, w, u, v)
println("address of w: ", pointer(w))
@. w = u + v
println("address of w: ", pointer(w))

address of w: Ptr{Float64} @0x000000000ea31c00
address of w: Ptr{Float64} @0x000000000ea7fe40
address of w: Ptr{Float64} @0x000000000eace080
address of w: Ptr{Float64} @0x000000000eace080
address of w: Ptr{Float64} @0x000000000eace080
address of w: Ptr{Float64} @0x000000000eace080
address of w: Ptr{Float64} @0x000000000eace080
address of w: Ptr{Float64} @0x000000000eace080


In [19]:
@benchmark w .= u .+ v setup=((u, v, w) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  15.847 μs … 448.334 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     30.573 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.645 μs ±  24.965 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▂▄▇█▇▅▃▃▂▂▃▃▃▃▄▃▂▂▁             ▁▁                        ▂
  ▄▆███████████████████████▇▇▇▇▆▇▇▇▇█████▇█▇▆▆▆▇▇▇▇▆▆▅▅▆▅▅▅▅▄▅ █
  15.8 μs       Histogram: log(frequency) by time       138 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
@benchmark broadcast!(+, w, u, v) setup=((u, v, w) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  13.498 μs …  5.633 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.231 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.968 μs ± 61.299 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▂▄▆██▆▄▃▂▃▂▂▃▃▃▂▁           ▁ ▁▁                         ▂
  ▆█████████████████████████▇▇▇▇████████▆▇▇▇█▇▆▇▆▆▅▅▅▆▆▅▄▁▃▃▅ █
  13.5 μs      Histogram: log(frequency) by time       143 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
@benchmark @.(w = u + v) setup=((u, v, w) = make_uvw($nx, $ny, $rng))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  17.203 μs … 445.992 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     29.271 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   37.076 μs ±  24.397 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▃▇██▅▄▃▂▂▂▃▂▂▃▂▂▁              ▁▁                         ▂
  ▄▆█████████████████████▇▇▇▆▅▅▅▆▆▇███▇█▆▇▇▇▇▇▆▆▅▆▆▆▅▆▃▄▄▄▂▄▅▄ █
  17.2 μs       Histogram: log(frequency) by time       135 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
(u, v, w) = make_uvw(nx, ny, rng)
println(pointer(u))
u_sub=view(u,2:nx-1, 2:ny-1)
println(pointer(w_sub))
u_sub

Ptr{Float64} @0x000000000a216980
Ptr{Float64} @0x000000000b726b08


198×198 view(::Matrix{Float64}, 2:199, 2:199) with eltype Float64:
 0.461395   0.440915    0.305823   …  0.0986442  0.998848   0.49889
 0.166002   0.300965    0.302738      0.203822   0.172654   0.493738
 0.382712   0.453751    0.298058      0.574655   0.666301   0.882695
 0.612983   0.294113    0.253554      0.0442611  0.19966    0.575853
 0.373337   0.659675    0.353996      0.106184   0.456294   0.153155
 0.58037    0.346601    0.79473    …  0.185008   0.948459   0.0867111
 0.0115094  0.0642434   0.77454       0.637316   0.415964   0.614659
 0.839676   0.135156    0.47844       0.0620136  0.234546   0.494745
 0.823588   0.735568    0.398121      0.0752732  0.169047   0.417675
 0.690916   0.595564    0.387696      0.876213   0.307443   0.517572
 0.0489413  0.511055    0.0364823  …  0.121591   0.327626   0.590291
 0.491618   0.982203    0.537727      0.693697   0.523378   0.421143
 0.837651   0.0453429   0.576436      0.319695   0.44656    0.0608431
 ⋮                                 

In [34]:
u

200×200 Matrix{Float64}:
 0.118689   0.273375   0.0996808   …  0.539537   0.479499   0.0197448
 0.811927   0.461395   0.440915       0.998848   0.49889    0.707127
 0.14515    0.166002   0.300965       0.172654   0.493738   0.58655
 0.900893   0.382712   0.453751       0.666301   0.882695   0.554739
 0.47388    0.612983   0.294113       0.19966    0.575853   0.769456
 0.801797   0.373337   0.659675    …  0.456294   0.153155   0.930386
 0.0909632  0.58037    0.346601       0.948459   0.0867111  0.300128
 0.815275   0.0115094  0.0642434      0.415964   0.614659   0.518083
 0.274359   0.839676   0.135156       0.234546   0.494745   0.401814
 0.989587   0.823588   0.735568       0.169047   0.417675   0.797965
 0.145137   0.690916   0.595564    …  0.307443   0.517572   0.116556
 0.531577   0.0489413  0.511055       0.327626   0.590291   0.530874
 0.560138   0.491618   0.982203       0.523378   0.421143   0.0753818
 ⋮                                 ⋱                        
 0.623975   0.72